#### Setup for Open CV endeavour, using predictions from file "for_anil.npz" (Image & predicted mask)
1. Setup : 
    conda install -c menpo opencv3; 
    pip install imutils (Note - conda install imutils fails)
2. OpenCV version: 3.1.0 used

3.  Main concern - predicted mask for size 64 has very low sum np.sum; need predicted mask for entire scan which can be loaded and use for OpenCV countour detection and finding centriod.

In [ ]:
import cv2 # import the opencv library
import imutils
print("Your OpenCV version: {}".format(cv2.__version__))

In [23]:
import pandas as pd
import numpy as np
import h5py
import pandas as pd
import argparse
import SimpleITK as sitk
from PIL import Image
import os, glob 
import os, os.path
import tensorflow as tf
import keras
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
from UNET_utils import *
from blob import *
%matplotlib inline

In [ ]:
data_dir = '../data/luna16/'
npz_prediction_file = 'subset2/for_anil.npz'

In [6]:
npz_dict = np.load(data_dir + npz_prediction_file)
print ("Keys in loaded dictionary : ")
npz_dict.keys()

Keys in loaded dictionary : 


['img1',
 'prediction_mask1',
 'prediction_featuremaps0',
 'img0',
 'prediction_featuremaps1',
 'prediction_mask0']

In [7]:
img1 = npz_dict.f.img1.reshape(64,64, 64)
predicted_mask1 = npz_dict.f.prediction_mask1.reshape(64,64, 64)
featuremaps1 = npz_dict.f.prediction_featuremaps1

img0 = npz_dict.f.img0.reshape(64,64, 64)
predicted_mask0 = npz_dict.f.prediction_mask0.reshape(64,64, 64)
featuremaps0 = npz_dict.f.prediction_featuremaps0

#### Discuss : 
1. Predicted mask sum is very low for input image ..?

In [8]:
rounded_predicted_mask1 = np.round(predicted_mask1)
print ("Predicted mask sum (Class1) : {}".format(np.sum(predicted_mask1)))
print ("Rounded Predicted mask sum (Class1): {}".format(np.sum(rounded_predicted_mask1)))

rounded_predicted_mask0 = np.round(predicted_mask0)
print ("\nPredicted mask sum (Class0): {}".format(np.sum(predicted_mask0)))
print ("Rounded Predicted mask sum (Class0): {}".format(np.sum(rounded_predicted_mask0)))

Predicted mask sum (Class1) : 3.1760988235473633
Rounded Predicted mask sum (Class1): 0.0

Predicted mask sum (Class0): 0.005565871950238943
Rounded Predicted mask sum (Class0): 0.0


In [9]:
def displaySlice(class_type, img, mask, sliceNo):
    plt.figure(figsize=[8,8]);    
    plt.subplot(121)
    plt.title('True Image ({})'.format(class_type))
    plt.imshow(img[:, :, sliceNo], cmap='bone');
    
    plt.subplot(122)
    plt.title('Predicted Mask ({})'.format(class_type))
    plt.imshow(mask[:, :, sliceNo], cmap='bone');
#     plt.axis("off")
    plt.show()

In [10]:
interact(displaySlice,class_type=fixed('Class1'),img=fixed(img1), \
         mask=fixed(predicted_mask1), sliceNo=(0,img1.shape[2]-1,1) );
# interact(displaySlice,class_type=fixed('Class0'),img=fixed(img0), \
#          mask=fixed(predicted_mask0), sliceNo=(0,img0.shape[2]-1,1) );

##### Discussion :
1. CV2: using Rounded_predicted_mask1....? 
2. Process 3D image in 2D slice's and then find countour in each 2D slice...? 
3. Convert image from BGR to RGB as OpenCV represents RGB images in reverse order by: 
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
4. Note the np.sum(predicted mask) is very low

In [11]:
def morphImage(img, kernel = np.ones((7,7), np.uint8)):
    img_dilated = cv2.dilate(img, kernel, iterations=1)
    img_eroded = cv2.erode(img_dilated, kernel, iterations=1)
    return (img_dilated, img_eroded)

In [12]:
# def findCentriod(cntrs, z_slice, MIN_THRESH = 7): #nodule area for diameter 3 is Pi*1.5*1.5 (i.e. 7) can be ignored
def findCentriod(cntrs, z_slice, MIN_THRESH = 0): #nodule area for diameter 3 is Pi*1.5*1.5 (i.e. 7) can be ignored    
    for c in cntrs:
        if cv2.contourArea(c) > MIN_THRESH:
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cZ = z_slice
            area = int(M["m00"])
            diameter = 2*(np.sqrt(area/np.pi))
#             writer.writerow(cX, cY, area, diameter)
            print(" Coord X = {}; Coord Y = {}; Coord Z = {}; Nodule Area = {}; Diameter = {} ".\
                  format(cX, cY, cZ, area, diameter))

In [13]:
def plot_morphImage(rslt):
    plt.figure(figsize=[8,8]);    
    plt.subplot(121)
    plt.title("Image after Dilation only");
    plt.imshow(rslt[0]);

    plt.subplot(122)
    plt.title("Image after Dilation & Erosion");
    plt.imshow(rslt[1]);

In [14]:
def process_mask(predicted_mask_tst):
    found_contour = False;
    for idx in range(predicted_mask_tst.shape[2]):
    # for idx in range(22, 40, 1):  
        one_slice = predicted_mask_tst[:, :, idx] 
        rslt = morphImage(one_slice)
        # plot_morphImage(rslt)

        tmp_slice = rslt[1]
        cv2.imwrite("tmp_slice.png", tmp_slice)
        image = cv2.imread("tmp_slice.png")

        # #AL....?
        # convert image from BGR to RGB as OpenCV represents RGB images in reverse order
        # cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        cntrs = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cntrs = cntrs[0] if imutils.is_cv2() else cntrs[1]
        if len(cntrs) > 0:
            found_contour = True;
        findCentriod(cntrs, idx)
        
    print("Found contour in mask : {}". format(found_contour))

##### Section A - testing with Simulated mask on image size (64,64,64) with Maks diameter = 12
1. Printing centriod coordinates X, Y , Z (as slice number)....?  

In [ ]:
predicted_mask_Simulated = makeMasks()[6].reshape(64,64,64)
process_mask(predicted_mask_Simulated)

##### Section B - using predicted mask for scan 
1. No contours found though slider show litte blips of 1's around slice#33 

In [ ]:
process_mask(rounded_predicted_mask1)

#### Result/Discussion on Erosion & Dilation operations
1. Perform dilation (to increase white region as we are getting very small sizes)
2. Then performing Erosion operation

##### Section C - findind centroid for 3D using blob method   
1. No 
https://github.com/kwohlfahrt/blob/blob/master/blob.py
blob.findBlobs(prediction_mask1[0,:,:,:,0].reshape(64,64,64), 
                    scales=range(2, 30), threshold=0.1)
Will return the pixel centers of a 3D blob.

In [32]:
predicted_mask_blob = makeMasks()[8].reshape(64,64,64)

(64, 64, 64)

In [31]:
findBlobs(predicted_mask_blob[:,:,:].reshape(64,64,64), 
                    scales=range(2, 30), threshold=0.1)

/Users/aluthra/Documents/DSE/ucsd-dse-capstone/Models/blob.py:46: RuntimeWarning: divide by zero encountered in true_divide
  / (12 * d)) * valid


array([[ 2, 32, 32, 32]])

In [36]:
findBlobs(rounded_predicted_mask1[:,:,:].reshape(64,64,64), 
                    scales=range(2, 30), threshold=0.1)

array([], shape=(0, 4), dtype=int64)